In [1]:
# Order preparation for NSE

# STATUS: Incomplete
# Run-time: 10 seconds

# Dependencies:
# /zdata/nse/*.pkl - for pickles generated by nse_ib program

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
# ib = IB().connect('127.0.0.1', 7496, clientId=11) # kavi tws live
ib = IB().connect('127.0.0.1', 4001, clientId=11) # kavi IBG live
# ib = IB().connect('127.0.0.1', 7499, clientId=11) # kavi tws paper-trade
# ib = IB().connect('127.0.0.1', 4004, clientId=11) # kavi IBG paper-trade

In [3]:
# %%time
# import os
import pandas as pd
import numpy as np
import datetime
from os import listdir

#... set limits and penalties
m_maxp = 1/100    # max margin allowed per scrip. e.g 1/100 = 1% of net liquidation value
min_rom = 3       # 8 would be 800% return

#... read the account info
ac = ib.accountSummary()
df_a = util.df(ac)

#... set max margin per position
net_liq = float(df_a[df_a.tag == 'NetLiquidation'].iloc[0].value)
max_p = net_liq*m_maxp

#... read the positions
ps = ib.portfolio()
df_p = util.df(ps)

df_p['ibSymbol'] = [s.symbol for s in df_p.contract.values]

#... get the lots and margins

# from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

# Rename Symbol and Margin fields
df_paisa = df_paisa.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# Convert columns to numeric and make margin to pct
df_paisa = df_paisa.apply(pd.to_numeric, errors='ignore')
df_paisa.marginpct = df_paisa.marginpct.div(100)

# Truncate to 9 characters for ibSymbol
df_paisa['ibSymbol'] = df_paisa.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MM', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_paisa.ibSymbol = df_paisa.ibSymbol.replace(ntoi)

df_slm = pd.merge(df_p, df_paisa[['ibSymbol', 'Mlot', 'TotMgnPerShr']])

#... make the blacklist

df1 = df_slm.groupby('ibSymbol').sum()
df1['used_margin'] = -df1.position * df1.TotMgnPerShr
df1['max_margin'] = max_p
df1['avail_margin'] = df1.position * df1.TotMgnPerShr + df1.max_margin

df1['max_units'] = (df1.avail_margin/(df1.Mlot*df1.TotMgnPerShr)).apply(np.floor)

blacklist = list(df1[df1.max_units <= 0].index)

#... build the high-pop-roc dataframe

# read pickles from zdata/nse sorted by rom
fs = listdir('./zdata/nse')
dfs = pd.concat([pd.read_pickle(r'./zdata/nse/'+f) for f in fs], axis=0).reset_index(drop=True).sort_values('rom', ascending=False)

# filter out puts which are not in blacklist
dfs_p = dfs[(dfs.right=='P') & (~dfs.ibSymbol.isin(blacklist))]

# filter out puts whose underlying price is greater than minimum rom
df2 = dfs_p[(dfs_p.rom > min_rom)]

# filter puts whose underlying price is greater than meanPrice
df3 = df2[(df2.undPrice > df2.meanPrice)]   # very very safe

In [62]:
df = df3.copy()   # make this the last dataframe to get the orders placed
contracts = [c.contract for c in df3.ticker]
orders = [LimitOrder(action='SELL', totalQuantity=lot, lmtPrice=expPrice) for lot, expPrice in zip(df3.lot, df3.expPrice)]

len(contracts)

33

In [ ]:
# Write closed orders placed to file
x  =  r'./zdata/' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_closing_orders.xlsx"
writer = pd.ExcelWriter(x)
df_p.to_excel(writer, 'all-options', index=False, header=1)
writer.save()